# Perfomance Analysis

In [23]:
import pandas as pd

## Sequential Model

In [24]:
LOG_FILE = '../log/seq.csv'

df = pd.read_csv(LOG_FILE)
df['phase'] = df.replace({'phase': {0: 'train', 1: 'val'}})['phase']
df.head(n=10)

,epoch,batch,batch_time(s),phase
0,0,0,25.979750,train
1,0,1,26.016988,train
2,0,2,26.434223,train
3,0,3,25.707898,train
4,0,4,25.592377,train
5,0,5,25.641662,train
6,0,6,25.612364,train
7,0,7,25.624064,train
8,0,8,25.591709,train
9,0,9,25.612150,train


In [25]:
df[df['phase'] == 'train']['batch_time(s)'].describe()

count    250.000000
mean      26.080359
std        0.344749
min       25.577825
25%       25.981775
50%       26.037149
75%       26.124101
max       28.670659
Name: batch_time(s), dtype: float64

In [26]:
df[df['phase'] == 'val']['batch_time(s)'].describe()

count    50.000000
mean     10.234476
std       0.191090
min      10.128167
25%      10.179445
50%      10.208933
75%      10.232612
max      11.475606
Name: batch_time(s), dtype: float64

In [34]:
df.groupby('epoch')['batch_time(s)'].sum().describe()

count       5.000000
mean     1406.362735
std         4.620646
min      1401.187116
25%      1402.878305
50%      1405.472299
75%      1410.945040
max      1411.330913
Name: batch_time(s), dtype: float64

## Pipeline Model